## NeuroTac Spiking Neural Network Demo

Intro paragraph or something here

In [1]:
# Import libraries for data collection
import sys
sys.path.append("..")
from utils.data_gatherer_neurotac import DataCollector

Here you can collect data from the neuroTac in real time

In [2]:
# Collection code
sensor_type = 'DAVIS240C_84010012'
path = "/home/farscope2/Documents/PhD/Lava_Demo/tests/"
collector = DataCollector(sensor_type)
data_path = collector.main(path=path)

In [3]:
print(data_path)

/home/farscope2/Documents/PhD/Lava_Demo/tests/DAVIS240C_84010012/data_gatherer_neurotac_11061641/events/taps_object_0_trial_0_events_on


After collecting the data, within this particular demo, the data must be preprocessed before being passed into our pretrained spiking neural network

In [1]:
from utils.data_processor import DataProcessor

sample_length = 3000
data = DataProcessor.load_data_np(f"{data_path}.npy")

if sensor_type == "DAVIS240C_84010012":
    data.pixel_reduction(40, 30, 20, 0)  # Crop data
else:
    data.pixel_reduction(160, 170, 60, 110)
data.remove_cuttoff(sample_length)
data.remove_duplicates()
data.threshold_pooling((4, 4), 4, 1)
input_tensor = data.create_lava_array(sample_length)

Below we will create our lava process components that are required to feed data into the network and visualise the output of the network.

In [ ]:
# Import all the components of our network
from components.visualiser import Visualiser
# from utils.utils import calculate_pooling_dim

# Import lava related libraries
# import numpy as np
from lava.magma.core.run_configs import Loihi2SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc import io
from lava.lib.dl import netx#, slayer

In [ ]:
# Import trained network model
net = netx.hdf5.Network(net_config="../networks/network.net")
# Create input ring buffer containing our input tensor we collected and preprocessed
source = io.source.RingBuffer(data=input_tensor)
# Output ring buffer to contain output spikes
sink = io.sink.RingBuffer(shape=(2,), buffer=sample_length)
# Create a visualiser object that will let us see network output in real-time
vis = Visualiser(in_shape=net.out.shape, sample_length=sample_length, window_size=50)

In [ ]:
# Connect processes
source.s_out.connect(net.inp)
net.out.connect(sink.a_in)
net.out.connect(vis.a_in)

In [ ]:
# Run the network for long enough to get all of the data through the network
run_condition = RunSteps(num_steps=sample_length)

# Map the defined encoder and adapter to their proc models
run_config = Loihi2SimCfg(select_tag='fixed_pt', select_sub_proc_model=True)

Run the network and view the moving window of output spikes

In [ ]:
# Run network
print("Running network")

source.run(condition=run_condition, run_cfg=run_config)
# Stop network execution
source.stop()